In [1]:
from importlib import reload
import flash.text.classification.data as data
from flash.text.classification import tokenizers as toks
from flash.text.classification import model
reload(data)
reload(model)
reload(toks)

<module 'flash.text.classification.tokenizers' from '/Users/49796/Documents/GitHub/lightning-flash/flash/text/classification/tokenizers/__init__.py'>

In [2]:
tok, vocab_size = toks.TEXT_CLASSIFIER_TOKENIZERS.get("prajjwal1/bert-tiny")(pretrained=True)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/tokenizers (https://github.com/huggingface/tokenizers).


In [27]:
from typing import Generator, List, Optional, Tuple, Union

import torch
from pytorch_lightning.utilities.exceptions import MisconfigurationException
from transformers import AutoConfig, AutoTokenizer

from flash.text.classification.tokenizers.base import BaseTokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Sequence, Digits, Whitespace
from tokenizers.normalizers import BertNormalizer
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer


class WhiteSpaceTokenizer(BaseTokenizer):

    def __init__(self, **backbone_kwargs):
        super().__init__("whitespace", False)

        self.max_vocab_size = backbone_kwargs.get("max_vocab_size", 50000)
        self.max_length = backbone_kwargs.get("max_length", 1000)
        self.batch_size = backbone_kwargs.get("batch_size", 1000)
        self.unk_token = backbone_kwargs.get("unk_token", "[UNK]")

        self.tokenizer = Tokenizer(WordLevel(unk_token=self.unk_token))
        self.tokenizer.pre_tokenizer = Sequence([Digits(), Whitespace()])
        self.tokenizer.normalizer = BertNormalizer()
        self.trainer = WordLevelTrainer(vocab_size=self.max_vocab_size, special_tokens=[self.unk_token])
 

    def fit(self, batch_iterator: Generator[List[str], None, None]) -> None:
        if self._is_fit:
            return
        self.tokenizer.train_from_iterator(batch_iterator, trainer=self.trainer)  # in-place
        self.vocab_size = self.tokenizer.get_vocab_size()  # get the actual vocab_size
        self._is_fit = True

    def __call__(
        self, x: Union[str, List[str]], return_tensors: Optional[str] = None, return_attention_mask: bool = False
    ) -> Union[List[int], torch.Tensor]:
        if not self._is_fit:
            raise MisconfigurationException("If pretrained=False, tokenizer must be fit before using it")

        output = {"input_ids": [x.ids for x in self.tokenizer.encode_batch(x)]}
        
        if return_attention_mask:
            output.update({"attention_mask": [x.attention_mask for x in self.tokenizer.encode_batch(x)]})

        # We’ll pad at the batch level.



        return self.tokenizer(
            x,
            return_token_type_ids=False,
            padding=True,  # pads to longest string in the batch, more efficient than "max_length"
            truncation=True,  # truncate to max_length supported by the model
            max_length=self.max_length,
            return_tensors=return_tensors,
        )

SyntaxError: invalid syntax (2951219036.py, line 48)

In [23]:
tok = WhiteSpaceTokenizer()

In [24]:
tok.voca

50000

In [25]:
tok.fit(["Hello, I am Pietro"])

In [26]:
tok.tokenizer.get_vocab_size()

6

In [44]:
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Sequence, Digits, Whitespace
from tokenizers.normalizers import BertNormalizer

In [92]:
unk_token = "[UNK]"
pad_token = "[PAD]"
tok = Tokenizer(WordLevel(unk_token=unk_token))
trainer = WordLevelTrainer(vocab_size=300, special_tokens=[pad_token, unk_token])
tok.pre_tokenizer = Sequence([Digits(), Whitespace()])
tok.normalizer = BertNormalizer()

In [93]:
example = "Hello, my name is Pietro and I am 19years old"
tok.train_from_iterator([example], trainer=trainer)

In [96]:
list(map(lambda x: x.ids, tok.encode_batch([example])))

[[6, 5, 4, 8, 2, 13, 7, 3, 9, 11, 12, 10]]

In [97]:
list(map(lambda x: x.attention_mask, tok.encode_batch([example])))

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [98]:
import transformers.tokenization_utils

In [99]:
from transformers import PreTrainedTokenizerFast

In [11]:
?WordLevel

Init signature: WordLevel(self, vocab, unk_token)
Docstring:     
An implementation of the WordLevel algorithm

Most simple tokenizer model based on mapping tokens to their corresponding id.

Args:
    vocab (:obj:`str`, `optional`):
        A dictionnary of string keys and their ids :obj:`{"am": 0,...}`

    unk_token (:obj:`str`, `optional`):
        The unknown token to be used by the model.
File:           ~/miniconda3/envs/flash-dev/lib/python3.8/site-packages/tokenizers/models/__init__.py
Type:           type
Subclasses:     


In [7]:
import tokenizers

In [3]:
# from csv
dm = data.TextClassificationData.from_csv(
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    input_fields="review",
    target_fields="sentiment",
    backbone=(tok, vocab_size),
    backbone_kwargs={"pretrained": True},
)

Using custom data configuration default-a4d22b10f26a6d19
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-3603c3c1f2d11b36.arrow
Using custom data configuration default-e799485285e8380f
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-e799485285e8380f/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/49796/.cache/huggingface/datasets/csv/default-e799485285e8380f/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-7350e04f1d315e84.arrow
Using custom data configuration default-791e28d155719303
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-791e28d155719303/

In [4]:
model = model.TextClassifier(dm.num_classes, backbone=dm.backbone)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).


In [5]:
model.predict(["hello"])

/Users/49796/Documents/GitHub/lightning-flash/flash/core/model.py:591: LightningDeprecationWarning: The `LightningModule.datamodule` property is deprecated in v1.3 and will be removed in v1.5. Access the datamodule through using `self.trainer.datamodule` instead.
  elif getattr(self, "datamodule", None) is not None:


TypeError: string indices must be integers

In [ ]:
from flash import Trainer
trainer = Trainer(max_epochs=1, limit_train_batches=2, limit_val_batches=2)
trainer.finetune(model, datamodule=dm, strategy="freeze_unfreeze")

In [ ]:
from flash.text.classification.data import TextDeserializer
deserializer = TextDeserializer(tok)
model.predict(["ciao"], deserializer=deserializer)